In [6]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 4.7 MB/s eta 0:00:00


In [1]:
import google.colab
google.colab.drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import torch

In [3]:
data_path = '/content/drive/MyDrive/AI ML/DACON HANSOL/data/'

In [4]:
train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

# train_ft = pd.read_csv(data_path + 'train_ft_97983.csv')
train_ft = pd.read_csv(data_path + 'train_120000_ver1.csv')

In [5]:
train_ft

,category,question,answer
0,건축구조,면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...
1,건축구조,내진설계의 종류 좀 알려줘,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다."
2,건축구조,철골구조의 장점이 뭐야?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...
3,건축구조,철골철근 콘크리트 구조가 뭐야?,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ..."
4,건축구조,철골구조는 어떤 방식이 있어?,철골구조는 일반철골구조와 경량철골구조가 있습니다.
...,...,...,...
119995,기타+시공,"새집증후군이 무엇이며, 외부에 소리가 안나가게 하는 목적으로 차음재만 설치해도 돼?","새집증후군이란 주택, 및 건축물 등 생활환경 속에서 일어나는 화학물질 과민증으로 신..."
119996,마감재+인테리어,"써모사이딩이 무엇이며, 인테리어 디자인에서 조명의 역할은 무엇인가요?",써모사이딩은 메탈사이딩이라고도 불리는 외장재이며 알루미늄 합금 강판에 고내후성 도료...
119997,마감하자,"훼손이 무엇이고, 공간 내 높은습도로 인해 도배지에 얼룩이 발생할 수 있어?","훼손은 터짐 하자, 몰딩수정 등 물리적인 충격이나 힘에 의해 발생하는 하자입니다. ..."
119998,기타+마감하자,"새집증후군의 원인이 무엇이고, 벽지에 반점이 생기는 원인이 뭐야?",새집증후군은 새솔 지은 집에 입주했을 때 이전에 없던 이상 증상이 신체에 나타나는 ...


In [10]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import pairwise_distances
embedding_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

def cosine_score(input_text, generated_text):
    input_embedding = embedding_model.encode(input_text)
    generated_embedding = embedding_model.encode(generated_text)

    cosine_similarity_score = cosine_similarity(input_embedding, generated_embedding)
    cosine_similarity_score = max(cosine_similarity_score, 0)

    return cosine_similarity_score

def evaluate_similarity(input_text, generated_text, alpha=0.95): # cosine 가중치 조절
    input_embedding = embedding_model.encode(input_text)
    generated_embedding = embedding_model.encode(generated_text)

    # 코사인 유사도
    cosine_sim = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0]

    # 자카드 유사도
    input_tokens = set(input_text.split())
    generated_tokens = set(generated_text.split())
    jaccard_sim = len(input_tokens.intersection(generated_tokens)) / len(input_tokens.union(generated_tokens))

    # 가중 평균 내보기
    weighted_sim = alpha*cosine_sim + (1 - alpha)*jaccard_sim

    return weighted_sim

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [11]:
question = train_ft['question'].tolist()[:200]
answer = train_ft['answer'].tolist()[:200]

In [13]:
similarity = [evaluate_similarity(q, a) for q, a in zip(question, answer)]

In [15]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.17.0-py3-none-any.whl size=34845034 sha256=dc5bc53927cbc3eeaa301195d3da8b7db32060de601f174ab25c218475da42a6
  Stored in directory: /root/.cache/pip/wheels/90/bb/ae/15e1cf26d17183040bac268fbf8d1ca1f45f7f4daa0fb76b91
Successfully built kiwipiepy-model


In [16]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 51.7 MB/s eta 0:00:00


In [17]:
from kiwipiepy import Kiwi

kiwi = Kiwi()
kiwi.tokenize(train_ft['question'][0])

[Token(form='면', tag='NNG', start=0, len=1),
 Token(form='지', tag='VV', start=1, len=1),
 Token(form='ᆫ', tag='ETM', start=1, len=1),
 Token(form='장치', tag='NNG', start=2, len=2),
 Token(form='가', tag='JKS', start=4, len=1),
 Token(form='뭐', tag='NP', start=6, len=1),
 Token(form='이', tag='VCP', start=7, len=0),
 Token(form='야', tag='EF', start=7, len=1),
 Token(form='?', tag='SF', start=8, len=1)]

In [18]:
from konlpy.tag import Hannanum

hannanum = Hannanum()
hannanum.pos(train_ft['question'][0])

[('면진장치', 'N'), ('가', 'J'), ('뭐', 'N'), ('이', 'J'), ('야', 'E'), ('?', 'S')]

In [23]:
hann_question = []
for i in question :
    pos_result = hannanum.pos(i)
    nouns = [word for word, pos in pos_result if pos.startswith('N')]
    hann_question.append(nouns)

In [24]:
hann_answer = []
for i in answer :
    pos_result = hannanum.pos(i)
    nouns = [word for word, pos in pos_result if pos.startswith('N')]
    hann_answer.append(nouns)

In [26]:
kiwi_question = []
for i in question :
    pos_result = kiwi.tokenize(i)
    nouns = [word for word, pos, _,_ in pos_result if pos.startswith('N')]
    kiwi_question.append(nouns)

In [27]:
kiwi_answer = []
for i in answer :
    pos_result = kiwi.tokenize(i)
    nouns = [word for word, pos, _,_ in pos_result if pos.startswith('N')]
    kiwi_answer.append(nouns)

In [33]:
hann_similarity = []
for i in range(200):
    sim = evaluate_similarity(' '.join(hann_question[i]), ' '.join(hann_answer[i]))
    hann_similarity.append(sim)

In [34]:
kiwi_similarity = []
for i in range(200):
    sim = evaluate_similarity(' '.join(kiwi_question[i]), ' '.join(kiwi_answer[i]))
    kiwi_similarity.append(sim)

In [36]:

print(sum(hann_similarity)/len(hann_similarity))
print(sum(kiwi_similarity)/len(kiwi_similarity))
print(sum(similarity)/len(similarity))

0.3999909680925762
0.4708368933948615
0.37356603705612346


In [54]:
for i in tqdm(range(200)) :
    a, b, c = question[i], answer[i], similarity[i]
    h, p, j = hann_question[i], hann_answer[i], hann_similarity[i]
    k, l, m = kiwi_question[i], kiwi_answer[i], kiwi_similarity[i]

    print(a)
    print('original',c,b)
    print('hannanum',j,h,p)
    print('kiwi',m,k,l)
    print()


  0%|          | 0/200 [00:00<?, ?it/s]

면진장치가 뭐야?
original 0.1462109714746475 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다.
hannanum 0.22086363434791564 ['면진장치', '뭐'] ['면진장치란', '지반', '진동', '에너지', '흡수', '건물', '진동', '진동', '격리장치']
kiwi 0.24698055452770656 ['면', '장치', '뭐'] ['면', '장치', '지반', '진동', '에너지', '흡수', '건물', '진동', '진동', '격리', '장치']

내진설계의 종류 좀 알려줘
original 0.41688823401927944 내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.
hannanum 0.5976887693007786 ['내진설계', '종류'] ['설계', '종류', '내진구조,', '진구조', '면진구조']
kiwi 0.692527858700071 ['내진', '설계', '종류'] ['내진', '설계', '종류', '내진', '구조', '저', '진', '구조', '면', '구조']

철골구조의 장점이 뭐야?
original 0.46369720101356504 철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가능한 것이 장점입니다.
hannanum 0.5752991345795718 ['철골구조', '장점', '뭐'] ['철골구조', '건물', '외벽', '하중', '때문', '고층', '건물', '것', '축', '가능', '것', '장점']
kiwi 0.60941224694252 ['철골', '구조', '장점', '뭐'] ['철골', '구조', '건물', '외벽', '하중', '때문', '고층', '건물', '건축', '가능', '것', '장점']

철골철근 콘크리트 구조가 뭐야?
original 0.5970305353403091 철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. 

In [29]:
hann_question

[['면진장치', '뭐'],
 ['내진설계', '종류'],
 ['철골구조', '장점', '뭐'],
 ['철골철근', '콘크리트', '구조', '뭐'],
 ['철골구조', '방식'],
 ['커튼월', '뭐'],
 ['내진구조', '뭐'],
 ['중목구조', '방식', '뭐'],
 ['기둥', '보', '구조', '방식', '뭐'],
 ['블록구조', '뭐'],
 ['철골구조', '단점', '뭐'],
 ['콘크리트', '구조', '방식'],
 ['철골콘크리트', '구조', '뭐'],
 ['면진구조', '뭐'],
 ['철골구조', '뭐'],
 ['면진구조', '진구조의', '차이점', '뭐'],
 ['철근콘크리트', '구조', '뭐'],
 ['건축물', '구조', '구분'],
 ['조적식', '구조', '방식'],
 ['목구조', '방식'],
 ['경량철골구조', '뭐'],
 ['석구조', '뭐'],
 ['화재', '취약', '철골구조', '보완법', '뭐'],
 ['프리케스트콘크리트', '구조', '뭐'],
 ['진구조', '뭐'],
 ['통나무구조', '방식', '뭐'],
 ['일반철골구조', '뭐'],
 ['내진설계', '안정성', '순서', '말'],
 ['경량목구조', '방식', '뭐'],
 ['조적식', '구조', '뭐'],
 ['벽돌구조', '뭐'],
 ['아세톤', '노출', '어디', '안좋'],
 ['공동주택', '실내', '라돈', '측정지점'],
 ['소화', '설치', '기준', '뭐'],
 ['아파트', '도배평수', '게산해'],
 ['방수', '발수', '차이', '뭐'],
 ['건설', '뭐'],
 ['토목', '뭐'],
 ['층간소음', '기준', '뭐'],
 ['흡음', '뭐'],
 ['새집증후군', '뭐'],
 ['라돈', '제외', '공동주택', '실내공기질', '측정지점'],
 ['사무실', '실내공기', '시료채취', '측정지점'],
 ['소화', '종류', '뭐'],
 ['새집증후군', '예방', '방법', '뭐'],
 [

'면진장치 뭐'

In [55]:
kiwi_question = []
for i in question :
    pos_result = kiwi.tokenize(i)
    nouns = [word for word, pos, _,_ in pos_result if pos.startswith('N')]
    kiwi_question.append(nouns)

def get_kiwi_nouns(text):
    pos_result = kiwi.tokenize(text)
    nouns = [word for word, pos, _,_ in pos_result if pos.startswith('N')]
    return ' '.join(nouns)

In [56]:

def get_kiwi_nouns(text):
    pos_result = kiwi.tokenize(text)
    nouns = [word for word, pos, _,_ in pos_result if pos.startswith('N')]
    return ' '.join(nouns)

def kiwi_evaluate_similarity(input_text, generated_text, alpha=0.95): # cosine 가중치 조절

    input_embedding = embedding_model.encode(get_kiwi_nouns(input_text))
    generated_embedding = embedding_model.encode(get_kiwi_nouns(generated_text))

    # 코사인 유사도
    cosine_sim = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0]

    # 자카드 유사도
    input_tokens = set(input_text.split())
    generated_tokens = set(generated_text.split())
    jaccard_sim = len(input_tokens.intersection(generated_tokens)) / len(input_tokens.union(generated_tokens))

    # 가중 평균 내보기
    weighted_sim = alpha*cosine_sim + (1 - alpha)*jaccard_sim

    return weighted_sim

In [57]:
kiwi_evaluate_similarity(question[0],answer[0])

0.23586944341659544

In [58]:
evaluate_similarity(question[0],answer[0])

0.1462109714746475

In [64]:
for i in range(100):
    print(question[i])
    print(answer[i])
    print('유사도', evaluate_similarity(question[i],answer[i]))
    print('키위 유사도',kiwi_evaluate_similarity(question[i],answer[i]))
    print()

면진장치가 뭐야?
면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다.
유사도 0.1462109714746475
키위 유사도 0.23586944341659544

내진설계의 종류 좀 알려줘
내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.
유사도 0.41688823401927944
키위 유사도 0.6710992872714996

철골구조의 장점이 뭐야?
철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가능한 것이 장점입니다.
유사도 0.46369720101356504
키위 유사도 0.5969122469425201

철골철근 콘크리트 구조가 뭐야?
철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. 철골은 강하고 가벼운 소재로, 높은 내구성과 강도를 가지고 있어 대규모 공간이나 강성이 요구되는 대형 고층건물에 적합합니다. 그러나 화재에 취약하고 녹이 슬면 강성이 떨어지는 단점이 있습니다. 이를 보완하기 위해 철근과 콘크리트를 섞어 사용하는 방식이 개발되었습니다. 철근콘크리트는 철근이 인장력을, 콘크리트가 압축력을 담당하여 강성을 가지며 콘크리트의 열전도가 낮은 재료상 특징을 이용해 서로의 단점을 보완하도록 결합한 구조입니다. 내화성, 내구성, 내진성능이 뛰어나지만 자중이 크고 시공과정이 복잡하며 공사기간이 길고 균일한 시공이 불가능하다는 단점이 있습니다. 철근철골콘크리트는 이 두 가지 구조의 장점을 살려 만든 것으로, 철골 뼈대로 구조를 세우고 그 주변을 철근과 콘크리트로 감싸거나, 철근과 콘크리트로 구조를 만들고 그 위에 철골을 올리는 형태로 만들어집니다. 이렇게 함으로써 철골의 내구성과 강도를 높이면서도 화재에 대한 안전성을 높일 수 있습니다.
유사도 0.5970305353403091
키위 유사도 0.5810824394226074

철골구조는 어떤 방식이 있어?
철골구조는 일반철골구조와 경량철골구조가 있습니다.
유사도 0.5056266997